# Download Sentinel-2 data

In this notebook we are going to download data (...).

In [1]:
%load_ext autoreload
%autoreload 2

%load_ext dotenv
%dotenv

Load the bounding box

In [2]:
import geopandas as gpd

boadella_bbox_gdf = gpd.read_file('data/boadella_bbox.geojson', crs='EPSG:4326')

boadella_bbox = list(boadella_bbox_gdf.geometry.total_bounds)
boadella_bbox

[2.792027806635944, 42.33057868499878, 2.838021549182864, 42.36457137143556]

Load the dates

In [10]:
import csv

dates = []
with open("data/dates.csv", "r") as file:
    reader = csv.reader(file)
    for row in reader:
        dates.append(row[0])
dates.sort()

dates[:5]

['2019-06-02', '2019-06-07', '2019-06-17', '2019-06-27', '2019-07-02']

We have the dates of available images as single days and we need them in format day, day (...)

In [11]:
time_interval = [(date, date) for date in dates[:5]]   # TODO put all dates
time_interval

[('2019-06-02', '2019-06-02'),
 ('2019-06-07', '2019-06-07'),
 ('2019-06-17', '2019-06-17'),
 ('2019-06-27', '2019-06-27'),
 ('2019-07-02', '2019-07-02')]

In [12]:
boadella_download_dict = {
    'Boadella': {
        'bounding_box': boadella_bbox,
        'time_interval': time_interval
    }
}

Connect to SH client (...)

In [13]:
from os import getenv
from eotdl.access import SHClient

sh_client_id = getenv('SH_CLIENT_ID')
sh_client_secret = getenv('SH_CLIENT_SECRET')

client = SHClient(sh_client_id=sh_client_id, 
                  sh_client_secret=sh_client_secret)

Let's download!

In [14]:
from eotdl.access import sentinel_2_download_parameters

sentinel_2_download_parameters.data_to_download = boadella_download_dict
sentinel_2_download_parameters.data_folder = 'data/sentinel_2'

process_requests = client.request_bulk_data(sentinel_2_download_parameters)

In [15]:
%%time

data = client.download_data(process_requests)

CPU times: user 208 ms, sys: 87.2 ms, total: 295 ms
Wall time: 13.6 s


Let's check the downloaded files

In [23]:
from glob import glob

rasters = glob('data/sentinel_2/*/*/*.tiff')
rasters[:5]

['data/sentinel_2/sentinel-2-l2a_Boadella_2019-06-07/90839abb69fd0df08bcc798c4b210006/response.tiff',
 'data/sentinel_2/sentinel-2-l2a_Boadella_2019-06-02/31065f0313941942895ce53ab4f9a2a3/response.tiff',
 'data/sentinel_2/sentinel-2-l2a_Boadella_2019-07-02/073923a2d94447815cf3099b06816205/response.tiff',
 'data/sentinel_2/sentinel-2-l2a_Boadella_2019-06-17/7aff418802685f538f0daab8ec5ef9b0/response.tiff',
 'data/sentinel_2/sentinel-2-l2a_Boadella_2019-06-27/219cbe573a0e901311771f20bbe1c050/response.tiff']

## Format downloaded data

In order to maintain a logic structure and ensure that the set is diregible by the EO-TDL environment, we must make sure that the project structure is compatible with the `STAC` convention, as we are going to generate the metadata of every downloaded item and collection.

In [1]:
from eotdl.curation import SHFolderFormatter

formatter = SHFolderFormatter('data/sentinel_2')
formatter.root

formatter.format_folders()

['data/sentinel_2/sentinel-2-l2a_Boadella_2019-06-07/90839abb69fd0df08bcc798c4b210006/response.tiff']
['data/sentinel_2/sentinel-2-l2a_Boadella_2019-06-02/31065f0313941942895ce53ab4f9a2a3/response.tiff']
['data/sentinel_2/sentinel-2-l2a_Boadella_2019-07-02/073923a2d94447815cf3099b06816205/response.tiff']
['data/sentinel_2/sentinel-2-l2a_Boadella_2019-06-17/7aff418802685f538f0daab8ec5ef9b0/response.tiff']
['data/sentinel_2/sentinel-2-l2a_Boadella_2019-06-27/219cbe573a0e901311771f20bbe1c050/response.tiff']
